# TRiP (Python version)

In [1]:
import numpy as np
import pandas as pd
from PIL import Image
from pathlib import Path
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
from scipy.signal import convolve2d


mpl.rcParams['figure.dpi']= 200 
plt.rcParams["axes.grid"] = False
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

/Users/joanbarreto/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def conv2(x, y, mode='same'):
    return np.rot90(convolve2d(np.rot90(x, 2), np.rot90(y, 2), mode=mode), 2)

# `crop_images`
Function that takes a a text file with two columns. First column is the plant ID, the second has the coordinates from ImageJ separate by one single space

In [9]:
def crop_images(crop_coords, images_path, img_extension):

    img_extension = '*.' + img_extension
    images = Path(images_path).glob(img_extension)
    images = [str(p) for p in images]
    
    # Create empty lists for ID (key) and value (coordinates)
    keys = []
    values = []
        
    # Gather IDs and coordinates
    for plant in range(len(crop_coords)):
        # Get ID
        plant_ID = crop_coords.iloc[plant,0]
        keys.append(plant_ID)
        # Get coordinates
        coords = crop_coords.iloc[plant,1]
        coords = coords.split(" ")
        coords = [int(i) for i in coords] # Convert values to integers
        values.append(tuple(coords))  # Append coords as tupple

    # Arrange them in a dictionary
    regions = dict(zip(keys, values))

    for i in images:
        # Read image and convert to array
        image = Image.open(i)
        image = np.array(image)

        # Extract and save the cropped images
        for plant, coords in regions.items():
            imj_col1 = coords[0]
            imj_row1 = coords[1]
            imj_col2 = imj_col1 + coords[2]
            imj_row2 = imj_row1 + coords[3]
            # Veryfy path exists, otherwise create it
            cropped_path = './cropped/' + plant + '/'
            cropped_path = os.path.dirname(cropped_path)
            if not os.path.exists(cropped_path):
                os.makedirs(cropped_path)
            # Define cropped image's full path
            cropped_path = cropped_path + '/crop_' + i.split('/')[-1]
            # Crop image
            cropped_image = image[imj_row1:imj_row2,imj_col1:imj_col2,:]
            sub_image = Image.fromarray(cropped_image)  # Convert to PIL format
            sub_image.save(cropped_path) # Save image
        
    print("All images saved in: " + images_path + "/cropped/")

In [12]:
%%time
# ~55sec for 380 images and 12 plants

# Read .txt coordinates as tab-separated in Pandas
crop_coords = pd.read_csv('./test/code/crop.txt', sep='\t')

# List images given path and extension
images_path = './test/input'
img_extension = 'JPG'

crop_images(crop_coords, images_path, img_extension)


All images saved in: ./test/input/cropped/
CPU times: user 52.5 s, sys: 638 ms, total: 53.1 s
Wall time: 53.3 s


# `space_time_deriv`

In [222]:
import numpy as np

# conv2 is meant to get the same results from conv2 in Matlab, which currently differ from convolve2d
def conv2(x, y, mode='same'):
    return np.rot90(convolve2d(np.rot90(x, 2), np.rot90(y, 2), mode=mode), 2)

def space_time_deriv(f):
    # N = f.shape[0]    
    # dims = f[0].shape
    f_temp = f
    N = len(f_temp)
    dims = f_temp[0]['im'].shape
    if N == 2:
        pre = np.array([0.5, 0.5])
        deriv = np.array([-1, 1])
    elif N == 3:
        pre = np.array([0.223755, 0.552490, 0.223755])
        deriv = np.array([-0.453014, 0.0, 0.453014])
    elif N == 4:
        pre = np.array([0.092645, 0.407355, 0.407355, 0.092645])
        deriv = np.array([-0.236506, -0.267576, 0.267576, 0.236506])
    elif N == 5:
        pre = np.array([0.036420, 0.248972, 0.429217, 0.248972, 0.036420])
        deriv = np.array([-0.108415, -0.280353, 0.0, 0.280353, 0.108415])
    elif N == 6:
        pre = np.array([0.013846, 0.135816, 0.350337, 0.350337, 0.135816, 0.01384])
        deriv = np.array([-0.046266, -0.203121, -0.158152, 0.158152, 0.203121, 0.046266])
    elif N == 7:
        pre = np.array([0.005165, 0.068654, 0.244794, 0.362775, 0.244794, 0.068654, 0.005165])
        deriv = np.array([-0.018855, -0.123711, -0.195900, 0.0, 0.195900, 0.123711, 0.018855])
    else:
        raise ValueError("No such filter size (N={})".format(N))

    fdt = np.zeros(dims)
    fpt = np.zeros(dims)
    for i in range(N):
        fdt += deriv[i] * f_temp[i]['im']
        fpt += pre[i] * f_temp[i]['im']
    
    pre = np.reshape(pre, (pre.shape[0],1)) # Reshape (N,1)
    deriv = np.reshape(deriv, (deriv.shape[0],1)) # Reshape (N,1)
    
    fx = conv2(conv2(fpt, pre, mode='same'), deriv, mode='same')
    fy = conv2(conv2(fpt, pre, mode='same'), deriv[::-1], mode='same')
    ft = conv2(conv2(fdt, pre, mode='same'), pre[::-1], mode='same')
    return fx, fy, ft


# `estimate_motion`

In [ ]:
import os
import cv2
import numpy as np

def estimate_motion(dirname):
    GRADIENT_THRESHOLD = 8
    DISPLAY = 0
    
    # load frames
    dirname = './cropped/plant07'
    frames = []
    ext = 'JPG'
    frames += [cv2.imread(os.path.join(dirname, f)) for f in os.listdir(dirname) if f.endswith(ext)]
    N = len(frames)
    print(f'loading {N} frames...')
                   
    # pre-processing
    scale = 60/max(frames[0].shape) # set scale (max dim = 60)
    frames = [cv2.resize(im, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC) for im in frames]
    f = [{'orig': im, 'im': cv2.cvtColor(im, cv2.COLOR_BGR2GRAY).astype(float)} for im in frames]
    ydim, xdim = f[0]['im'].shape
    
    # compute motion
    print('computing motion...')
    taps = 7
    blur = [1, 6, 15, 20, 15, 6, 1]
    blur = np.array(blur) / np.sum(blur)
    s = 1 # sub-sample spatially by this amount
    N = len(f) - (taps-1)
    Vx = np.zeros((ydim//s, xdim//s, N))
    Vy = np.zeros((ydim//s, xdim//s, N))
    for k in range(N):
        fx, fy, ft = space_time_deriv(f[k:k+taps])
        fx2 = cv2.filter2D(fx*fx, -1, blur, borderType=cv2.BORDER_REPLICATE)
        fy2 = cv2.filter2D(fy*fy, -1, blur, borderType=cv2.BORDER_REPLICATE)
        fxy = cv2.filter2D(fx*fy, -1, blur, borderType=cv2.BORDER_REPLICATE)
        fxt = cv2.filter2D(fx*ft, -1, blur, borderType=cv2.BORDER_REPLICATE)
        fyt = cv2.filter2D(fy*ft, -1, blur, borderType=cv2.BORDER_REPLICATE)
        grad = np.sqrt(fx**2 + fy**2)
        grad[:, :5] = 0
        grad[:5, :] = 0
        grad[:, xdim-4:xdim] = 0
        grad[ydim-4:ydim, :] = 0
                
        # Compute optical flow
        cx = 0
        bad = 0
        for x in range(0, xdim, s):
            cy = 0
            for y in range(0, ydim, s):
                M = np.array([[fx2[y, x], fxy[y, x]], [fxy[y, x], fy2[y, x]]])
                b = np.array([fxt[y, x], fyt[y, x]])
                if np.linalg.cond(M) > 1e2 or grad[y, x] < GRADIENT_THRESHOLD:
                    Vx[cy, cx, k] = 0
                    Vy[cy, cx, k] = 0
                    bad += 1
                else:
                    v = np.linalg.inv(M) @ b
                    Vx[cy, cx, k] = v[0]
                    Vy[cy, cx, k] = v[1]
                cy += 1
            cx += 1

        if bad / (xdim * ydim) == 1:
            print(f"WARNING on frame {k}: no velocity estimate")


In [ ]:
# import libraries
from pathlib import Path
from PIL import Image
import os
import sys
import pandas as pd
import numpy as np
import time


# Print user's arguments
arg1 = str(sys.argv[1])  # images_path
arg2 = str(sys.argv[2]) # cropfile_path
arg3 = str(sys.argv[3]) # img_extension
print("\nImages folder: ", arg1)
print("Path to crop file: ", arg2)
print("Images' extension: .", arg3)
print("\n")

# Define `crop_all` function
def crop_all(images_path, cropfile_path, img_extension):

    img_extension = '*.' + img_extension
    images = Path(images_path).glob(img_extension)
    images = [str(p) for p in images]
    images.sort()

    # Read crop file
    crop_coords = pd.read_csv(cropfile_path, sep='\t', header=None)

    # Create empty lists for ID (key) and value (coordinates)
    keys = []
    values = []

    # Gather IDs and coordinates
    for plant in range(len(crop_coords)):
        # Get ID (path to each 'cropped' folder)
        plant_ID = crop_coords.iloc[plant,0] #
        plant_ID = plant_ID.split(" ")[0] # Split by " "; take the first
        keys.append(plant_ID) # Add this to the 'keys' list
        # Get coordinates
        coords = crop_coords.iloc[plant,0]
        coords = coords.split(" ")[1:]  # This must change if using \t sep!!
        coords = [int(i) for i in coords] # Convert values to integers
        values.append(tuple(coords))  # Append coords as tupple

    # Arrange keys and values in a dictionary
    regions = dict(zip(keys, values))

    ti = len(images)  # total images (ti)
    ci = 0  # current image (ci)

    # Loop thorugh each original image
    for i in images:
        # Current image
        ci = ci+1
        print(f'Processing image {ci}/{ti}: {i}')
        # Read image and convert to array
        image = Image.open(i)
        image = np.array(image)
        # Extract and save the cropped images (assuming imagej's format)
        for plant, coords in regions.items():
            imj_col1 = coords[0] # vertical left
            imj_row1 = coords[1] # hortizontal bottom
            imj_col2 = imj_col1 + coords[2] # vertical right
            imj_row2 = imj_row1 + coords[3] # horizontal top
            # Veryfy path exists, otherwise create folder
            cropped_path = '../cropped/' + plant + '/'  # path
            cropped_path = os.path.dirname(cropped_path) # convert to string
            if not os.path.exists(cropped_path):  # verify if exist
                os.makedirs(cropped_path)
            # Define cropped image's full path
            cropped_path = cropped_path + '/crop_' + i.split('/')[-1]
            # Crop image
            cropped_image = image[imj_row1:imj_row2,imj_col1:imj_col2,:] # crop
            sub_image = Image.fromarray(cropped_image)  # Convert to PIL format
            sub_image.save(cropped_path) # Save image

    # Print where images were saved
    print("\nAll folders were saved in: ../cropped/")




# Define input variables
images_path = arg1
crop_coords = arg2
img_extension = arg3

# Execute function
start_time = time.time() # #tart timer
crop_all(images_path, crop_coords, img_extension) # Crop images
end_time = time.time()  # End timer
total_time = round(end_time - start_time,2)
print("\nExecution time: ", total_time, " seconds\n")




# Instructions
# 1. Make sure you have a folder "input" with images, and another "code" with the code.
# 2. Open the terminal and set "code" as current directory:
#### cd code
# 3. Execute the python script providing three arguments separated by comma: input path; crop file; image extension. These arguments should be placed after calling python (python) and the cropping function (crop_all.py):
#### python crop_all.py ../input C1_crop.txt JPG
